In [27]:
import numpy as np
import scipy.stats as st
import cvxpy as cvx
import pandas as pd
import datetime

In [22]:
tickers=pd.read_csv('tickers.csv')
tickers=tickers['ticker'].to_list()
data = pd.read_csv('./data/'+tickers[0]+'.csv')
data = data.drop(['Open', 'High','Low','Close','Adj Close','Volume'], axis=1)
for ticker in tickers:
    d = pd.read_csv('./data/'+ticker+'.csv')
    d = d.drop(['Open', 'High','Low','Adj Close','Volume'], axis=1)
    d = d.rename(columns={'Close': ticker})
    data = pd.merge(data,d,on='Date')

In [36]:
cal = data[['Date']]
cal['Datetime'] = pd.to_datetime(cal['Date'])
cal['year'] = cal['Datetime'].dt.year
cal['month'] = cal['Datetime'].dt.month
cal['week'] = cal['Datetime'].dt.week

/tmp/ipykernel_1188/3635024390.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal['Datetime'] = pd.to_datetime(cal['Date'])
/tmp/ipykernel_1188/3635024390.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal['year'] = cal['Datetime'].dt.year
/tmp/ipykernel_1188/3635024390.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [40]:
cal['mon'] = 1
for index, row in cal.iterrows():
    if index > 0 and cal.iloc[index,3] != cal.iloc[index-1,3]:
        cal.iloc[index,5] = cal.iloc[index-1,5]+1
    else:
        cal.iloc[index,5] = cal.iloc[index-1,5]

/tmp/ipykernel_1188/501106492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal['mon'] = 1
/home/nske/miniconda3/envs/jp38/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [41]:
data = pd.merge(data,cal,on='Date')

In [46]:
data_smp = data.query('year < 2018')
data_smp = data_smp.set_index('Date')
data_smp = data_smp.drop(['Datetime','year','month','week','mon'], axis=1)

In [55]:
R = data_smp.pct_change()
R = R.dropna()
# R = R.asfreq(pd.infer_freq(R.index))

In [57]:
T = R.shape[0]
N = R.shape[1]

In [63]:
Benchmark = pd.read_csv('sp500.csv')
Benchmark = pd.merge(Benchmark,cal,on='Date')
Benchmark = Benchmark.query('year < 2018')
Benchmark = Benchmark.set_index('Date')
Benchmark = Benchmark.drop(['Open', 'High','Low','Adj Close','Volume','Datetime','year','month','week','mon'], axis=1)
Benchmark = Benchmark.pct_change()
Benchmark = Benchmark.dropna()

In [66]:
weight = cvx.Variable(N)
error = cvx.Variable(T)
tracerr = cvx.sum_squares(error)
asset_sqt = R/np.sqrt(T)
index_sqt = Benchmark/np.sqrt(T)

In [76]:
asset = asset_sqt.values[0:T,:]
indx = index_sqt.values[0:T,:]

In [77]:
mintracerr = cvx.Problem(cvx.Minimize(tracerr), [indx-asset*weight == error,cvx.sum_entries(weight) == 1.0,weight >= .0])

/home/nske/miniconda3/envs/jp38/lib/python3.8/site-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 4 times so far.

  warnings.warn(msg, UserWarning)


ValueError: Cannot broadcast dimensions  (1258, 1) (1258,)